# Cournot duopoly

**Context:** there a re two companies producing the same identical good for the same group of costumers. In particular, for each time istant each company has to decide how much of that good produce. Conversely at each time instant the group of consumer has to decide weather to buy or not as well as from which of the two company, eventually. Compines strive for maximization of profit.

**Model:** the istance has been approached from the active inference perspective. Companies are modeled as agents which:
- at each time step decide the quantity ($q_i$) of goods to produce 
- production costs $C(q_i)$ are fixed
- Costumers decide the price according to a profit function $P(Q=q_1+q_2)$, which is unkown to the agents

**Further assumptions:** these are minor assumptions in the code:
- the total number of costumers is unkown to companies and fixed over time [variante: $~N(\mu,\sigma)$ per dare varianza all'agente]
- the amount of good sold by the first company is handled through a "reputation" parameter ($\in[0,1]$), higher it is higher the amount of good sold. The second company has (1-reputation) consumers.

**Active Inference perspective:** What follow is a brief recap of the agents and environment design. The first observation to be made is about multy-agency perspective in active inference paradigm. Not having found examples about this specific case I imagined two agents acting through the same set of possible action in the same environment, in the same moments. In this view lies also the choice of the "Cournot Duopoly" example.
Furthermore, for every company:

- observation O={profit}; [attualmente sono sold_items le osservazioni... il ricavo è variabile... pensarci su]
- states S={sold_item}; 
- actions U={how much do I produce};

- Likelihood matrix A assumes a perfect perception:
- Transition matric B assumes to sell less or equal to the amount produced, in an uninformative way; [magari considerare una campana]
- Goal oriented priors C assumes the basic "the more I sell, the more I gain", assuming a selling price; [pensarci seriamente...]
- Prior over stated D uninformative, I expect to see the market.

Finally, the environment takes in input the amount produced by the two companies, elaborates the market price [che non viene mai usato!!!] and assigns to costumers the items according to the reputation. The output of the environment are sales data for both company. 



In [175]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [176]:
import pymdp
from pymdp import utils

## Problem initialization

Dimensionalities of the hidden states factors, the control factors, and observation modalities

In [ ]:
# Sales states (how much has been sold)
Sales_1 =  np.array([0, 1, 2, 3, 4, 5])
Sales_2 =  np.array([0, 1, 2, 3, 4, 5])

# Market states (if I have produced enaugh or not)
Market_1 = ['Produced excess', 'Produced right', 'Not produced enaugh']
Market_2 = ['Produced excess', 'Produced right', 'Not produced enaugh']

# number of states
n_states_1 = [len(Sales_1), len(Market_1)]
n_states_2 = [len(Sales_2), len(Market_2)]

# number of factors
n_factors_1 = len(n_states_1) #? capiamo
n_factors_2 = len(n_states_2)

# actions
Production_1 =  np.array([0, 1, 2, 3, 4, 5])
Production_2 =  np.array([0, 1, 2, 3, 4, 5])

satisfy_market_action = ['Market survey']

# observations of sales and market observations (satisfied costumer demand or not)
obs_sales_1 = np.array([0, 1, 2, 3, 4, 5])
obs_sales_2 = np.array([0, 1, 2, 3, 4, 5])

obs_market_1 = ['Exceded', 'Satisfied', 'Unsatisfied'] 
obs_market_2 = ['Exceded', 'Satisfied', 'Unsatisfied']

# number of observations and modalities
num_obs_1 = [len(obs_sales_1), len(obs_market_1)]
num_obs_2 = [len(obs_sales_2), len(obs_market_2)]

num_modalities_1 = len(num_obs_1) 
num_modalities_2 = len(num_obs_2)

## Agents initialization

### Likelihood matrices

In [207]:
A1 = utils.obj_array(num_modalities_1)
A2 = utils.obj_array(num_modalities_2)

Sales modality:

In [208]:
# Oss.Vendite x State Sales x State Market
# Tuttavia assumento perfetta percezione delle vendite il market non influisce.
A1_sales = np.zeros((len(obs_sales_1), len(Sales_1), len(Market_1)))
A2_sales = np.zeros((len(obs_sales_2), len(Sales_2), len(Market_2)))

# modeling perfect perception of the sold items indipendently from the market state:
for i in range(len(Market_1)):
    A1_sales[:, :, i] = np.eye(len(Sales_1))

for i in range(len(Market_2)):
    A2_sales[:, :, i] = np.eye(len(Sales_2))


In [209]:
A1[0] = A1_sales
A2[0] = A2_sales

Market modality:

In [210]:
A1_market = np.zeros((len(obs_market_1), len(Sales_1), len(Market_1)))
A2_market = np.zeros((len(obs_market_2), len(Sales_2), len(Market_2)))

trust_market = 1.0 # 1 means perfect perception

# Assign perception probabilities
A1_market[0, :, 0] = trust_market           # exceded <- produced excess
A1_market[1, :, 0] = (1 - trust_market)/2   # satisfied <- produced excess
A1_market[2, :, 0] = (1 - trust_market)/2   # etc...
A1_market[0, :, 1] = (1 - trust_market)/2
A1_market[1, :, 1] = trust_market
A1_market[2, :, 1] = (1 - trust_market)/2
A1_market[0, :, 2] = (1 - trust_market)/2
A1_market[1, :, 2] = (1 - trust_market)/2
A1_market[2, :, 2] = trust_market

A2_market[0, :, 0] = trust_market
A2_market[1, :, 0] = (1 - trust_market)/2
A2_market[2, :, 0] = (1 - trust_market)/2
A2_market[0, :, 1] = (1 - trust_market)/2
A2_market[1, :, 1] = trust_market
A2_market[2, :, 1] = (1 - trust_market)/2
A2_market[0, :, 2] = (1 - trust_market)/2
A2_market[1, :, 2] = (1 - trust_market)/2
A2_market[2, :, 2] = trust_market

In [211]:
A1[1] = A1_market
A2[1] = A2_market

### Transition matrices

In [212]:
B1 = utils.obj_array(n_factors_1)
B2 = utils.obj_array(n_factors_2)

B sales context:

In [213]:
B1_sales = np.zeros((len(Sales_1), len(Sales_1), len(Production_1)))
B2_sales = np.zeros((len(Sales_2), len(Sales_2), len(Production_2)))

In [214]:
#completing agent1 transition matrix
#for i in range(len(Production_1)):  # for every level of production 
for i in range(len(Production_1)):
    for start_state in range(len(Sales_1)):  # Initial states of sales
        max_sales = min(start_state, i)  # maximum sale is min(start_state, i)
        
        if max_sales >= 0:
            B1_sales[:max_sales+1, start_state, i] = 1 / (max_sales + 1)  # uninformative distr.

#completing agent2 transition matrix
#for i in range(len(Production_2)):  # for every level of production
for i in range(len(Production_2)):
    for start_state in range(len(Sales_2)):  # Initial states of sales
        max_sales = min(start_state, i)  # maximum sale is min(start_state, i)
        
        if max_sales >= 0:
            B2_sales[:max_sales+1, start_state, i] = 1 / (max_sales + 1)  # uninformative distr.

In [215]:
B1[0] = B1_sales
B2[0] = B2_sales

B market context:

In [216]:
B1_market = np.zeros((len(Market_1), len(Market_1), len(satisfy_market_action)))
B2_market = np.zeros((len(Market_2), len(Market_2), len(satisfy_market_action)))

#Adesso implemento con eye perchè la mia azione "do nothing" non può incidere sullo stato, ma boh capiamo
B1_market[:,:,0] = np.eye(len(Market_1))
B2_market[:,:,0] = np.eye(len(Market_2))

In [217]:
B1[1] = B1_market
B2[1] = B2_market

### Goal oriented priors 

In [218]:
C1 = utils.obj_array_zeros(num_obs_1)
C2 = utils.obj_array_zeros(num_obs_2)

Profit oriented goals:

In [219]:
consumer_price_1 = 10 # earning from a single piece
production_price_1 = 5 # price of a single piece
discount_rate_1 = 1 # the more I produce the less I pay

C1[0] = np.array((consumer_price_1 - production_price_1 * discount_rate_1) * Production_1)
#mmmmh se la scrivo come funzione con consmer_price = market price? Capiamo

In [220]:
consumer_price_2 = 10 # earning from a single piece
production_price_2 = 5 # price of a single piece
discount_rate_2 = 1 # the more I produce the less I pay

C2[0] = np.array((consumer_price_2 - production_price_2 * discount_rate_2) * Production_2)

Market oriented goals:

In [221]:
C1_market = np.zeros(len(obs_market_1))
C1_market[0] = 0
C1_market[1] = 10 
C1_market[2] = -2

C1[1] = C1_market

In [222]:
C2_market = np.zeros(len(obs_market_2))
C1_market[0] = 0
C1_market[1] = 10 
C1_market[2] = -2

C2[1] = C2_market

### States prior

In [223]:
D1 = utils.obj_array(n_factors_1)
D2 = utils.obj_array(n_factors_2)

# Sales state prior
D1[0] = np.full(len(Sales_1), 1 / len(Sales_1))
D2[0] = np.full(len(Sales_2), 1 / len(Sales_2))

# Market state prior
D1[1] = np.full(len(Market_1), 1 / len(Market_1))
D2[1] = np.full(len(Market_2), 1 / len(Market_2))

### Agents construction

In [224]:
from pymdp.agent import Agent

my_agent1 = Agent(A = A1, B = B1, C = C1, D = D1)
my_agent2 = Agent(A = A2, B = B2, C = C2, D = D2)

## Generative process

We shall create an environment able to elaborate the actions of the two agents and return an observation for both.

Even if this is a first attempt and the agent expect only to see sales results, I'm already introducing the price function which determines the market price of the good given the joint production of the two agents. This is for sure to be integrated somehow somewhere.

In [225]:
class CournotMarket(object):

    def __init__(self, reputation=0.5, total_demand=5, max_customer_price=10, sensibility=1):
        self.reputation = float(reputation)
        self.total_demand = float(total_demand)
        self.max_customer_price = float(max_customer_price)
        self.sensibility = float(sensibility)
        self.market_obs_names = ['Satisfied', 'Exceded', 'Unsatisfied']
    
    def market_status(self, actual_production, preferred_demand):
        if actual_production > preferred_demand:
            return 'Exceded'
        elif actual_production < preferred_demand:
            return 'Unsatisfied'
        else:
            return 'Satisfied'

    def step(self, action1, action2):
        total_production = action1[0] + action2[0]
        total_sell = min(self.total_demand, total_production)
        
        market_price = max(0, self.max_customer_price - self.sensibility * total_production)
        
        preferred_demand_1 = int(self.total_demand * self.reputation)
        preferred_demand_2 = self.total_demand - preferred_demand_1
        
        actual_sales_1 = min(action1[0], preferred_demand_1)
        actual_sales_2 = min(action2[0], preferred_demand_2)
        
        remaining_demand_1 = preferred_demand_1 - actual_sales_1
        remaining_demand_2 = preferred_demand_2 - actual_sales_2
        
        if remaining_demand_1 > 0 and action2[0] > actual_sales_2:
            extra_sales_2 = min(action2[0] - actual_sales_2, remaining_demand_1)
            actual_sales_2 += extra_sales_2
            remaining_demand_1 -= extra_sales_2
        
        if remaining_demand_2 > 0 and action1[0] > actual_sales_1:
            extra_sales_1 = min(action1[0] - actual_sales_1, remaining_demand_2)
            actual_sales_1 += extra_sales_1
            remaining_demand_2 -= extra_sales_1
        
        obs_sales = [int(actual_sales_1), int(actual_sales_2)] 
        # obs_market = ['Exceded' if actual_sales_1 > preferred_demand_1 else ('Unsatisfied' if actual_sales_1 < preferred_demand_1 else 'Satisfied'),
        #             'Exceded' if actual_sales_2 > preferred_demand_2 else ('Unsatisfied' if actual_sales_2 < preferred_demand_2 else 'Satisfied')]
        obs_market = [self.market_status(action1[0], preferred_demand_1),
                      self.market_status(action2[0], preferred_demand_2)]
        return obs_sales, obs_market

## Active Inference Loop

Now let's write a function that takes the:
- Agent
- Environment 
- Time length
and run the active inference loop

In [226]:
def run_active_inference_loop(my_agent1, my_agent2, my_env, T = 5):
    """Initialize first observation"""
    market_label = ['Exceded' ,'Satisfied', 'Unsatisfied']
    obs1 = [5, obs_market_1.index(market_label[2])] #sales at T = -1 + market satisfaction <= come se dovessimo ancora produrre
    obs2 = [3, obs_market_2.index(market_label[2])]

    for t in range(T):
        qs_1 = my_agent1.infer_states(obs1)
        qs_2 = my_agent2.infer_states(obs2)

        q_pi_1, efe_1 = my_agent1.infer_policies()
        q_pi_2, efe_2 = my_agent2.infer_policies()

        action_1 = my_agent1.sample_action()
        action_2 = my_agent2.sample_action()

        obs = my_env.step(action_1, action_2)
        obs1 = [obs[0][0], obs_market_1.index(obs[1][0])]
        obs2 = [obs[0][1], obs_market_1.index(obs[1][1])]

        print(f'Production industry 1 at time {t}: {action_1[0]}')
        print(f'Production industry 2 at time {t}: {action_2[0]}') 
        print(f'Sells of industries at time {t}: {obs[0]}')
        print(f'Market at time {t}: {obs[1]}')


## Main

In [227]:
import warnings
warnings.filterwarnings('ignore')

In [228]:
reputation_env = 0.5
total_demand_env = 4
env = CournotMarket(reputation = reputation_env, total_demand = total_demand_env, max_customer_price = 100, sensibility = 1)

T = 5

run_active_inference_loop(my_agent1, my_agent2, env, T = T)

Production industry 1 at time 0: 5.0
Production industry 2 at time 0: 3.0
Sells of industries at time 0: [2, 2]
Market at time 0: ['Exceded', 'Exceded']
Production industry 1 at time 1: 5.0
Production industry 2 at time 1: 3.0
Sells of industries at time 1: [2, 2]
Market at time 1: ['Exceded', 'Exceded']
Production industry 1 at time 2: 2.0
Production industry 2 at time 2: 2.0
Sells of industries at time 2: [2, 2]
Market at time 2: ['Satisfied', 'Satisfied']
Production industry 1 at time 3: 2.0
Production industry 2 at time 3: 2.0
Sells of industries at time 3: [2, 2]
Market at time 3: ['Satisfied', 'Satisfied']
Production industry 1 at time 4: 2.0
Production industry 2 at time 4: 2.0
Sells of industries at time 4: [2, 2]
Market at time 4: ['Satisfied', 'Satisfied']
